<h1 style='font-size:40px;font-weight:bold;margin-bottom:10px;'> Boilerplate - Looker Projects </h1> <br>
<p style='font-size:20px;'> codigo generico para tableros de PRODUCTIVIDAD DE FLOTA</p>

# 1. Libraries

In [ ]:
from datetime import datetime, date
from datetime import timedelta, time
import pickle
import numpy as np
import pandas as pd
import os
import json
import base64
import math
from google.cloud import bigquery
from google.oauth2 import service_account
from sqlalchemy import create_engine

import sys
sys.path.append('../../')
from functions.genericFunctions import *
from functions.functionEncription import *
urlConnections = '../../settings'

# 2. Functions

In [ ]:
def list_diff(list1, list2):
    out = [item for item in list1 if not item in list2]
    return out

def getLastData(imeisToAnalyze,project,clientId,nameDate,tableName):
    # Last date
    try:  
        deviceList = pd.read_sql("""select imei, max(max_date) as begin_time from p_confd 
                where type='channel_daily' and flat=1 and process_name='{}' and client_id={} and dashboard_type='{}'
                group by imei """.format(processName, clientId, dashboardType), psqlCon)
        print('postgres ', deviceList.shape)
        if len(deviceList)==0: 
            
            queryLastDate = clientBigQueryLookerAnalytics.query("""SELECT imei, max({}) as begin_time
                                              FROM `{}.{}.{}`
                                              where clientId='{}'
                                              GROUP BY imei""")
            resultLastDate = queryLastDate.result()
            deviceList = resultLastDate.to_dataframe()
            print('looker ', deviceList.shape)
            
        else: pass
        deviceList['begin_time'] = pd.to_datetime(deviceList.begin_time).dt.tz_localize(None)
        
    except: 
        deviceList = pd.DataFrame(columns={'imei', 'begin_time'})            

    totalImeiList = imeisToAnalyze.imei.unique().tolist()
    currentDeviceList = deviceList.imei.unique().tolist()
    imeiDiff = list_diff(totalImeiList, currentDeviceList)
    
    dateToProcess = datetime.utcnow() + timedelta(days = -5)  
    startDate = datetime.combine(dateToProcess, time.min)
    endDate = datetime.combine(datetime.utcnow(), time.min) 
    deviceList['end_time'] = endDate
    for imei in imeiDiff:
        plate = pd.DataFrame({
        'imei': imei,
        'begin_time': startDate,
        'end_time': endDate,
        }, index = { 'indice': 0 })
        deviceList = deviceList.append(plate)
 
    deviceList['end_time'] = deviceList['end_time'].apply(lambda x: x.replace(microsecond = 0))
    deviceList.reset_index(drop=True, inplace=True)
    return (deviceList)

def saveData(finalDataFrame,clientId,env,whatToDo,tableName,credenciales):
    if len(finalDataFrame.index) > 0:
        finalDataFrame.to_gbq(destination_table = "{}.{}".format(clientId,tableName), 
                project_id = env, credentials = credenciales, if_exists = whatToDo)
    else:
        print("No hay data, no se guarda nada")      

In [ ]:
def idleTime(data):
    ignition=1
    timeSpeed = 0
    minSpeed=2
    event = 0
    data= data.reset_index(drop=True)
    previousRow = data.index[0]
    
    for row in data.index[range(1, data.shape[0])]:   
        speed = data.at[row, "speed"]
        generateTime = data.at[row, "generateTime"] 
        generateTimePrevious = data.at[previousRow,"generateTime"] 
        ignitionState = data.at[previousRow,"ignitionState"] 
        
        try:
            if math.isnan(speed) == True:
                speed = 0
            if (speed < minSpeed) & (ignitionState==ignition):
                timeSpeed = timeSpeed + (generateTime - generateTimePrevious).seconds
                event = event + 1
            previousRow = row
        except Exception as ex:
            continue

    return(timeSpeed)

def movementTime(data):
    ignition=1
    timeSpeed = 0
    minSpeed=2
    event = 0
    data= data.reset_index(drop=True)
    previousRow = data.index[0]
    
    for row in data.index[range(1, data.shape[0])]:   
        speed = data.at[row, "speed"]
        generateTime = data.at[row, "generateTime"] 
        generateTimePrevious = data.at[previousRow,"generateTime"] 
        ignitionState = data.at[previousRow,"ignitionState"] 
        
        try:
            if math.isnan(speed) == True:
                speed = 0
            if (speed >= minSpeed) & (ignitionState==ignition):
                timeSpeed = timeSpeed + (generateTime - generateTimePrevious).seconds
                event = event + 1
            previousRow = row
        except Exception as ex:
            continue

    return(timeSpeed)

In [ ]:
def historyLevelData(deviceList,unit):
    historyTempData = pd.DataFrame()
    history=pd.DataFrame()
    
    for imei in deviceList['imei'].unique():
        startDate = deviceList.loc[deviceList['imei']==imei, "begin_time"].max()
        endDate = deviceList.loc[deviceList['imei']==imei, "end_time"].max()

        try:
            
            queryHistory = clientBigQueryLocationWorld.query("""SELECT *
                                    FROM `conf_idelcial_BQ.{}`
                                    WHERE generateTime >= '{}' and generateTime <= '{}'
                                    order by generateTime""".format(imei, startDate, endDate))

            resultHistory = queryHistory.result()
            history = resultHistory.to_dataframe()
            history = history[(history['_t']=='OnlineMessage')]
            history['imei']=imei
        
        except Exception as e:
            print(e)
            continue
            
        if history.shape[0]>0:
            dataFrame = pd.merge(history, unit, how='inner', on=['imei'])    
            dataFrame = dataFrame.sort_values(by = ["imei", "generateTime"])
            dataFrame["generateTime"] = pd.to_datetime(dataFrame.generateTime).dt.tz_localize(None)
            dataFrame["generateTime"] = dataFrame["generateTime"] + timedelta(hours = offset)
            dataFrame["mileage"]=dataFrame["mileage"].apply( lambda x: format(x,'.2f')).values.astype(np.float)
            dataFrame["additionalTripId"]=dataFrame["additionalTripId"].astype(int)
            dataFrame['date'] = dataFrame.generateTime.dt.date
            dataFrame['hour'] = dataFrame.generateTime.dt.hour
            dataFrame = dataFrame.sort_values(by = ["imei","generateTime"],ascending=False)
            dataFrame["km"]=dataFrame["mileage"]-dataFrame["mileage"].shift(-1)
            dataFrame.driverFullName =dataFrame.driverFullName.fillna('Sin Información')
            dataFrame=dataFrame.fillna(0)

            dataFrame['test'] = dataFrame.date.astype(str) + dataFrame.hour.astype(str) + dataFrame.groupName.astype(str) + dataFrame.automotor_plate.astype(str) + dataFrame.imei.astype(str) + dataFrame.alias.astype(str) + dataFrame.additionalTripId.astype(str) + dataFrame.driverFullName.astype(str)
            dataFrame.test=dataFrame.test.astype(str)
            dataFrame=dataFrame.fillna(0)
            
            final=pd.DataFrame()

            for trip in dataFrame.additionalTripId.unique():
                temp=dataFrame[dataFrame.additionalTripId==trip]

                if temp.shape[0]<2:
                    cualquier_valor=2
                else:
                    groupData = temp.sort_values('generateTime', ascending=True).groupby(['date', 'hour', 'groupName', 'automotor_plate','automotor_chasis', 'imei', 'alias', 'additionalTripId', 'driverFullName'])
                    dataInfo = temp.sort_values('generateTime', ascending=True)[['date', 'hour', 'groupName', 'automotor_plate','automotor_chasis', 'imei', 'alias', 'additionalTripId', 'driverFullName','test']].drop_duplicates()
                    dataInfo['hour_mileage'] = groupData.km.sum().to_list()
                    dataInfo['odometer'] = groupData.mileage.max().to_list()

                    final=final.append(dataInfo)

            if final.shape[0]>0:

                dataFrame=dataFrame[dataFrame.additionalTripId.isin(final.additionalTripId.unique())].sort_values('generateTime')

                df_group = pd.DataFrame(dataFrame.sort_values('generateTime', ascending=True).groupby(['test','additionalTripId'])['speed'].sum()).reset_index()


                for t in dataFrame.test.unique():
                    aux=dataFrame[dataFrame.test==t]
                    aux_index=aux.index.values

                    # calculos para idle - movement
                    timeSpeed_idle=idleTime(aux)
                    timeSpeed_move=movementTime(aux)

                    aux_final=df_group[df_group.test==t].index.values

                    df_group.at[aux_final,'idleTime']=timeSpeed_idle
                    df_group.at[aux_final,'movementTime']=timeSpeed_move

                df_group=df_group.sort_values('test').reset_index(drop=True)
                final=final.sort_values('test').reset_index(drop=True)

                final['idleTime']=df_group.idleTime
                final['movementTime']=df_group.movementTime

    #             final=final.sort_values(by='hour')
                stop = (final.groupby(['date', 'hour', 'groupName', 'automotor_plate','automotor_chasis', 'imei', 'alias', 'driverFullName'])['idleTime', 'movementTime'].sum()).reset_index()

                stop['stopTime'] = 3600 - (stop.idleTime + stop.movementTime)
                stop['additionalTripId']=-1

                dataInfo = pd.merge(final, stop[['imei', 'date', 'hour', 'stopTime']], how='left', on=['imei', 'date', 'hour'])
                aux = pd.merge(final, stop[['imei', 'date', 'hour', 'stopTime']], how='left', on=['imei', 'date', 'hour']).drop_duplicates(subset=('imei', 'date', 'hour', 'stopTime'),keep='first').reset_index()

                stop=stop.sort_values(by=['imei', 'date', 'hour', 'stopTime']).reset_index(drop=True)
                dataInfo=dataInfo.sort_values(by=['imei', 'date', 'hour', 'stopTime']).reset_index(drop=True)
                aux=aux.sort_values(by=['imei', 'date', 'hour', 'stopTime']).reset_index(drop=True)

                final['stopTime']=0

                c=0
                index_dejar=aux['index'].unique()
                for index in index_dejar:
                    final.loc[index,'stopTime']=stop.loc[c,'stopTime']
                    c=c+1    
                historyTempData=historyTempData.append(final)
        
    return (historyTempData.reset_index(drop=True))

# funciones de guardado

In [ ]:
def tripsProductivityUpload(tripsData,businessId):    
    tempUpload = (tripsData.groupby(['imei']).generateTime.max()).reset_index().rename(columns={'generateTime':'max_date'})
    tempUpload['date'] = datetime.today().strftime('%Y-%m-%d')
    tempUpload['type'] = 'channel_trips'
    tempUpload['flat'] = 1
    tempUpload['client_id'] = businessId
    tempUpload['process_name'] = processName
    tempUpload['dashboard_type'] = dashboardType
    tempUpload.to_sql('looker_data_upload', con=psqlCon, schema='platform' ,if_exists='append', index=False)
    return(tripsData)
    
def dailyProductivityUpload(dailyData,businessId):
    tempUpload = (dailyData.groupby(['imei']).generateTime.max()).reset_index().rename(columns={'generateTime':'max_date'})
    tempUpload['date'] = datetime.today().strftime('%Y-%m-%d')
    tempUpload['type'] = 'channel_daily'
    tempUpload['flat'] = 1
    tempUpload['client_id'] = businessId
    tempUpload['process_name'] = processName
    tempUpload['dashboard_type'] = dashboardType
    tempUpload.to_sql('looker_data_upload', con=psqlCon, schema='platform' ,if_exists='append', index=False)
    return(dailyData)

def saveTripsData(businessId,tripsData,tableName,project,credenciales):
    tripsSave = pd.read_sql("""select imei, max(max_date) as begin_time from plat""", psqlCon)
    tripsSaveData = pd.DataFrame()

    for imei in tripsData.imei.unique():
        tripDate = tripsSave[tripsSave.imei==imei]
        if len(tripDate)!=0:
            tripsSaveData = tripsSaveData.append(tripsData[(tripsData.imei==imei) &
                                                           (tripsData.generateTime > tripDate.begin_time.max())])
       
        else : tripsSaveData = tripsSaveData.append(tripsData[(tripsData.imei==imei)])
    tripsSaveData.to_gbq(destination_table = "business_{}.{}")
    return(tripsData)

def saveDailyData(businessId,dailyData,tableName,project,credenciales):
    dailySave = pd.read_sql("""select  """, psqlCon)

    dailySaveData = pd.DataFrame()
    
    for imei in dailyData.imei.unique():
        dailyDate = dailySave[dailySave.imei==imei]
        if len(dailyDate)!=0:
            dailySaveData = dailySaveData.append(dailyData[(dailyData.imei==imei) &
                                                           (dailyData.generateTime > dailyDate.begin_time.max())])

        else : dailySaveData = dailySaveData.append(dailyData[(dailyData.imei==imei)])
    dailySaveData.to_gbq(destination_table = "business_{}.{}")
    return(dailyData)

# funciones propias de productividad


In [ ]:
def tripsTotalProcess(trips1):
    #Es necesario tener 4 dias para sacar el promedio de km recorrido por vehiculo.
    filteredData=historyTempData.copy()
    filteredData.additionalTripId=filteredData.additionalTripId.astype(object)
    temp=filteredData[["groupName","imei","unitAlias","additionalTripId","brand", "model","devicePlate",'km']].groupby(["groupName","additionalTripId","imei","unitAlias","brand", "model","devicePlate"]).mean()
    temp=temp.reset_index(level=[0,1,2,3,4,5,6]).rename(columns={'km':'km_covered_per_day_by_trip'})
    km_covered=temp.loc[:,['imei',"additionalTripId",'km_covered_per_day_by_trip']].rename(columns={'additionalTripId':'tripId'})

    #Los calculos de aqui en adelante solo considerean el ultimo dia de extraccion.
    km_total=pd.DataFrame()
    filteredData=analyzeData[["groupName","imei","unitAlias","additionalTripId","brand", "model","devicePlate",'mileage']]
    for imei  in filteredData.imei.unique():
        temp=filteredData[filteredData.imei==imei]
        temp['km_total_per_day_by_trip']=temp.mileage.max()-temp.mileage.min()
        temp=temp.drop_duplicates(subset=["additionalTripId",'km_total_per_day_by_trip'],keep='last')
        km_total=km_total.append(temp)
    km_total=km_total.loc[:,['imei',"additionalTripId",'km_total_per_day_by_trip']].rename(columns={'additionalTripId':'tripId'})
    tripsFinal=trips1.merge(km_total,on=['imei','tripId'],how='left')
    tripsFinal=tripsFinal.merge(km_covered,on=['imei','tripId'],how='left')
    return tripsFinal

def tripsCalculation(startDate):
    trips1=hoursTripsProcess(analyzeData,startDate)
    tripsF=tripsTotalProcess(trips1)
    tripFinal2=startEndTrips()
    tripsTotal=tripsF.merge(tripFinal2,on=['imei','tripId'])
    
    chofer=analyzeData[['imei','unitAlias','group_name','driverFullName']].drop_duplicates().rename(columns={'group_name':'groupName'})
    tripsTotal=tripsTotal.merge(chofer,on=['imei','unitAlias','groupName'],how='inner')    
    return tripsTotal

# 3. Connection settings

In [ ]:
 ### CONFIDENCIAL

# 4. Parameters

In [ ]:
parameters = { "_id": "geuv8sncw1yMCKPTy0Ge","businessId": 89,"clientIdList":[],"userId": 0,"timeZone": -5 }

In [ ]:
env='CONFIDENCIAL#

In [ ]:
project=env
nameDate='generateTime'
tableName='tabla_confidencial_1'

dashboardType="confidencial_standard"
lookerModel="modelo_confidencial"
processName="productivity"

minVel = 2
businessId   = parameters['businessId']
clientListParameter = parameters['clientIdList']
offset = int(parameters['timeZone'])

print(businessId,offset,lookerModel,dashboardType,processName)

# 5. Requirements verification

In [ ]:
clientsList = pd.read_sql("""select  wk.imei, wk.business_id, client_id
                                from [sql_conf_tabl].[dbo].[conf_wa] wk
                                inner join [sql_conf_tabl].[dbo].[conf_un] wu
                                on wk.imei=wu.imei
                                where wk.business_id={} 
                                and wk.status_id=3 
                                order by client_id""".format(businessId), sqlServerCon)

print(clientsList.client_id.nunique(),clientsList.imei.nunique())
clientsList

In [ ]:
if len(tuple(clientListParameter))>0:
    clientListF=clientsList[clientsList.client_id.isin(clientListParameter)]
    print('cinco primeros vehiculos del cliente:',clientListF.head())
    if clientListF.shape[0]==0:
        print('los clientID ingresados no existen se procede a tomar todos')
        clientListF=clientsList.copy()
    print(clientListF.shape)
else:
    clientListF=clientsList.copy()
    
    print('se toman todos los cleintes')

In [ ]:
imeisToAnalyze=pd.DataFrame()

for clientId in clientListF.client_id.unique():
    print(clientId)
    try:
        unit = decryptCatalogs(urlConnections,clientId, "device")
        unit = unit[['imei','client','country', 'automotor_plate','automotor_chasis','alias', 'group_name']]

        try:
            if unit.shape[0]>0:
                activeDevices, numActiveDevices = True, unit.shape[0]
                imeis=unit.copy()
                imeis['clientId']=clientId
                imeisToAnalyze=imeisToAnalyze.append(imeis)
        except :
            print('clientId no existe en encriptacion')
    except :
        print('1')

print('# clientes',imeisToAnalyze.clientId.nunique(),'y # vehiculos',imeisToAnalyze.imei.nunique())
print('nombres clientes :',imeisToAnalyze.client.unique())

# 6. Data Processing

In [ ]:
processBeginTime = datetime.now()

if (activeDevices == True): 
    for clientId in imeisToAnalyze.clientId.unique():
        
        daily_infoByImei=pd.DataFrame() 
        trips_infoByImei=pd.DataFrame()
        
        tempClient=imeisToAnalyze[imeisToAnalyze.clientId==clientId]
        
        deviceList = getLastData(tempClient, project,clientId,nameDate,tableName)
        print('-------------------------------------------------------------------')
        print('tempClient',tempClient.shape,'clientID', clientId,'vehiculos a analizar : ',deviceList.shape)
        
        for imei in deviceList.imei.unique():

            tempDeviceList=deviceList[deviceList.imei==imei] 
            
            tripsTotal = historyLevelData(tempDeviceList,tempClient)

            if tripsTotal.shape[0]>0:
                
                tripsTotal['clientId']=clientId.astype(str)
                tripsTotal['clientName']=tempClient.reset_index(drop=True).loc[0,'client']
                print(tempClient.reset_index(drop=True).loc[0,'client'])

                tripsTotal.rename(columns={'date':'generateTime', 'hour':'hourPerDay', 'additionalTripId':'tripId', 'automotor_plate':'devicePlate','hour_mileage':'hourMileage',
                                  'alias':'unitAlias', 'idle_time':'idleTime', 'automotor_chasis': 'automotorChasis'},inplace=True)    
                tripsTotal = tripsTotal[tripsTotal.hourMileage>=0]
                tripsTotal = tripsTotal[(tripsTotal.idleTime + tripsTotal.movementTime + tripsTotal.stopTime)>0]

                tripsTotal = tripsTotal.drop_duplicates()

                dailyHourReport = (tripsTotal.groupby(['generateTime', 'hourPerDay','clientId','clientName', 'groupName', 'devicePlate', 'imei',
                                   'unitAlias','automotorChasis', 'driverFullName'])[['hourMileage','idleTime', 'stopTime', 'movementTime']].sum()).reset_index()
                dailyHourReport['odometer'] = list(tripsTotal.groupby(['generateTime', 'hourPerDay','clientId','clientName', 'groupName', 'devicePlate', 'imei',
                                                   'unitAlias', 'automotorChasis','driverFullName']).odometer.max())

                # validar que el groupName este en object para el esquema de la tabla
                dailyHourReport.groupName=dailyHourReport.groupName.apply(lambda x : 'Sin información' if x == 0 else x)
                dailyHourReport.groupName=dailyHourReport.groupName.astype(str)

                tripsTotal.groupName=tripsTotal.groupName.apply(lambda x : 'Sin información' if x == 0 else x)
                tripsTotal.groupName=tripsTotal.groupName.astype(str)

                tripsTotal.hourMileage = tripsTotal.hourMileage.astype('float')
                tripsTotal.odometer = tripsTotal.odometer.astype('float')
                tripsTotal.idleTime = tripsTotal.idleTime.astype('float')
                tripsTotal.stopTime = tripsTotal.stopTime.astype('float')
                tripsTotal.movementTime = tripsTotal.movementTime.astype('float')
                
                tripsTotal.hourPerDay=tripsTotal.hourPerDay.astype(int)
                dailyHourReport.hourPerDay=dailyHourReport.hourPerDay.astype(int)

                dailyHourReport.hourMileage = dailyHourReport.hourMileage.astype('float')
                dailyHourReport.odometer = dailyHourReport.odometer.astype('float')
                dailyHourReport.idleTime = dailyHourReport.idleTime.astype('float')
                dailyHourReport.stopTime = dailyHourReport.stopTime.astype('float')
                dailyHourReport.movementTime = dailyHourReport.movementTime.astype('float')

                dailyHourReport["generateTime"] = dailyHourReport["generateTime"].astype('datetime64[ns]') + timedelta(hours = -offset*2)
                tripsTotal["generateTime"] = tripsTotal["generateTime"].astype('datetime64[ns]') + timedelta(hours = -offset*2)
            
                dailyHourReport['generateTime']=dailyHourReport.generateTime.apply(lambda x : datetime(x.year, x.month,x.day))
                tripsTotal['generateTime']=tripsTotal.generateTime.apply(lambda x : datetime(x.year, x.month,x.day))

                daily_infoByImei=daily_infoByImei.append(dailyHourReport) 
                trips_infoByImei=trips_infoByImei.append(tripsTotal)

            print(imei,tripsTotal.shape,daily_infoByImei.shape,trips_infoByImei.shape,'wow')
            print('--------------------')  
        
        ### SAVE DATA
        crdl=bigQueryCredentialsLookerAnalytics
        ver='confidencial'
        
        if (trips_infoByImei.shape[0]>0):
            trips_infoByImei=trips_infoByImei.drop(columns='test')
            trips_infoByImei.tripId=trips_infoByImei.tripId.astype(int)
            tripsData = saveTripsData(businessId, trips_infoByImei, 'modelo_confidencial_daily_trip_hour', ver,crdl)
            tripsData = tripsProductivityUpload(tripsData,businessId)

        if (daily_infoByImei.shape[0]>0):
            dailyData = saveDailyData(businessId, daily_infoByImei, 'tabla_confidencial_1',ver,crdl)
            dailyData = dailyProductivityUpload(dailyData,businessId)

print(' horas:',daily_infoByImei.shape,'| trips:',trips_infoByImei.shape)
#print(datetime.now())
processEndTime = datetime.now()

# 7. Save execution log

In [ ]:
#humidityNullValues, productivityNullValues, productivityOutlier, humidityOutlier, tempMinOutlier, tempMaxOutlier, tempMeanOutlier, humMinOutlier, humMaxOutlier, humMeanOutlier = dataframeExplore(analysisData)
#logDataframe = pd.DataFrame({
#    'clientId' : clientId,
#    'process' : processName,
#    'dashboardType' : dashboardType,
#    'lookerModel' : lookerModel,
#    'startAt' : processBeginTime,
#    'endedAt' : processEndTime,
#    'rowsGeneratedHistory' : historyTempData.shape[0],
#    'rowsGeneratedTrips' : tripsData.shape[0],
#    'rowsGeneratedDaily' : dailyData.shape[0],
#    'processed_devices' : historyTempData.imei.nunique()
#}, index = {1})